# Spinning Earth due to an Asteroid Impact

ModSim Project 3

Kristtiya Guerra, HK Rho

In [ ]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim library
from modsim import *
import random
import decimal

In [ ]:
#Earth is spinning(stationary) and the asteroid impacts at known place&angle
#sweep through the size of the asteroid changing the spin 
#should I go over the derivative of angular rotation?????

#speed and mass of the asteroid determines the effect it will cause on Earth's rotation

# 1. choose a place to observe the time (maybe at the equator?)
# 2. 

In [ ]:
"""
Function that converts:
Diameter of the asteroid -> Volume of the asteroid
Volume of the asteroid -> Mass of the asteroid
"""

def D_to_mass(D_asteroid, system):
    unpack(system)
    
    volume_asteroid = 4/3 * pi * (D_asteroid/2)**3
    m_asteroid = density_asteroid * volume_asteroid
    
    return m_asteroid

In [ ]:
def force_to_torque():
    
    
    
    return torque_on_earth

In [ ]:
# And an inition condition (with everything in SI units)

Vol_earth = 6.371e6 # m
dD = 10 # m in diameter
D_asteroid = linrange(50, 5000, dD, endpoint = True) # m in diameter

sweepMass = SweepSeries()
sweepMass[D_asteroid] = Vol_to_mass(D_asteroid, system) #convert this to mass and store it as mass

init = State(m_asteroid = sweepMass[0]
             rotation_speed = 16/36 # km/s
             )

In [ ]:
# Making a system object

system = System(m_earth = 5.972e24, # kg
                density_asteroid = 2000, # kg / m^3
                v_asteroid = 25000 # m/s
                
    
    
                init=init,
                r_final=r_asteroid + r_earth,
                t_0=0 # s,
                t_end=1e9 # s)